# Lecture 5 Attacks on Blockchain

### Imports

In [57]:
#Import statements
from IPython.display import HTML, display
import hashlib as hasher
from draw_blockchain import show

### Hash function

In [58]:

def hashbits(input):
    hash_obj = hasher.sha256()
    inputbytes = input.encode()
    hash_obj.update(inputbytes)
    hashbytes = hash_obj.digest()
    return ''.join(f'{x:08b}' for x in hashbytes)

def hash(input):
    hash_obj = hasher.sha256()
    inputbytes = input.encode()
    #print(type(inputbytes))
    hash_obj.update(inputbytes)
    return hash_obj.hexdigest()

def numberOfInitZeros(hashStr):
  count = 0
  for i in range (0 , len(hashStr)):
    if hashStr[i] == '1':
      break
    count += 1
  return count

### Block

In [59]:
class Block:
    def __init__(self, data, creator=None, previous=None, nonce=0):
        self.data = data
        if previous is None:
            self.previous = None
            self.previous_hash = ""
            self.creator = Miner(0 , "0")
            self.height = 0
            self.color = "#AEF751"
        else:
            self.previous = previous
            self.previous_hash = previous.hash
            self.creator = creator
            self.height = previous.height+1
            self.color = creator.color
        self.nonce = nonce
        self.hash = self.hash_block()
        self.children = []

    def hash_block(self):
        return hashbits(self.data+ self.creator.name + self.previous_hash + str(self.nonce))

    def print(self):
          print(self.data + " "+ self.creator.name + " " + str(self.height))

### Blockchain

In [60]:
import random

blockReward = 10
uncleReward = 5

class Blockchain:
    def __init__(self, genesis_data, difficulty):
        self.chain = []
        self.chain.append(Block(genesis_data))
        self.difficulty = difficulty
        self.size = 0
        self.uncles = []

    def longestChain(self):
      max = self.chain[0].height
      for block in self.chain:
        if block.height > max:
          max = block.height
      maxes = [block for block in self.chain if block.height == max]
      r = random.choices(maxes, k=1)
      return r[0]
        
    def add(self, newBlock):
        self.chain.append(newBlock)
        newBlock.previous.children.append(newBlock)
        self.size +=1
        if self.inlongestChain(newBlock):
          newBlock.creator.reward += blockReward
          if self.uncles and newBlock.height > self.uncles[0].height:
            self.referenceUncle()
        else:  
          self.uncles.append(newBlock)
        
    def print(self):
      for block in self.chain:
        block.print()
        print("________")

    def hasFork(self):
      for block1 in self.chain:
        for block2 in self.chain:
          if block1!=block2 and block1.height == block2.height:
            return True
      return False

    def checkMiner(self, miner):
      last = self.longestChain()
      count = 0
      while last!=None:
        if last.creator == miner:
          count += 1
        last = last.previous
      return count

    def referenceUncle(self):
      uncleblock = self.uncles[-1]
      uncleblock.creator.reward += uncleReward
      self.uncles.pop()
    
    def inlongestChain(self, block):
          last = self.longestChain()
          while last!= None:
                if last == block: #need more of a check for if fork
                      # print("last",last.height,"block", block.height)
                      return True
                last = last.previous
          return False

## Miners

### Normal miner

In [61]:
class Miner:
  def __init__(self, miningPower, name, blockchain=None):
    self.miningPower = miningPower
    # self.nonce = random.randint(0,100000) #original one
    self.nonce = 0
    self.name = name
    self.blockchain = blockchain
    self.reward = 0
    self.color = "#7EDBF6"
    if self.blockchain != None:
      self.lastBlock = blockchain.longestChain()
  
  def UpdateLast(self):
    latest = self.blockchain.longestChain()
    if latest.height > self.lastBlock.height:
        self.lastBlock = latest

  def PoWSolver(self):
    for i in range (0 , self.miningPower):
      newBlock = Block(str(self.blockchain.size), self, self.lastBlock, self.nonce)
      h = newBlock.hash_block()
      count = numberOfInitZeros(h)
      if count >= bc.difficulty:
        bc.add(newBlock)
        self.lastBlock = newBlock
      self.nonce += 1


### Selfish miner

In [62]:

class SelfishMiner(Miner):
  def __init__(self, miningPower, name, blockchain=None):
    super().__init__(miningPower, name, blockchain)
    self.privateBlocks = []
    self.publishNext = False
    self.color = "#F59AEE"

  def UpdateLast(self):
    #add this function
    latest = self.blockchain.longestChain()
    publicheight = latest.height
    if publicheight > self.lastBlock.height:
        self.privateBlocks = []
        self.lastBlock = latest
        self.publishNext = False
    if publicheight == self.lastBlock.height-1 and len(self.privateBlocks)> 1:
        for block in self.privateBlocks:
            self.blockchain.add(block)
            self.privateBlocks = []
    if publicheight == self.lastBlock.height:
        for block in self.privateBlocks:
            self.blockchain.add(block)
            self.privateBlocks = []
            self.publishNext = True

  def PoWSolver(self):
    #add this function
    for i in range (0 , self.miningPower):
      newBlock = Block(str(self.blockchain.size), self, self.lastBlock, self.nonce)
      h = newBlock.hash_block()
      count = numberOfInitZeros(h)
      if count >= bc.difficulty:
        if self.publishNext:
            self.blockchain.add(newBlock)
            self.publishNext = False
        else:
            self.privateBlocks.append(newBlock)
        self.lastBlock = newBlock
      self.nonce += 1

In [63]:
#Main
bc = Blockchain("0" , 11)
miners = []
for i in range(15):
  m = Miner(5 ,"m"+str(i), bc)
  miners.append(m)
selfish = SelfishMiner(7, "selfish", bc)
while bc.size < 30:
  selfish.PoWSolver()

  for m in miners:
    m.PoWSolver()

  selfish.UpdateLast()

  for m in miners:
    m.UpdateLast()

print(bc.hasFork())
print(bc.checkMiner(selfish))
print(bc.longestChain().height)
print("Fraction {}".format(bc.checkMiner(selfish) /bc.longestChain().height ))
total = selfish.miningPower
for m in miners:
    total += m.miningPower
print("alpha {}".format(selfish.miningPower/ total))
for min1 in miners:    
  print(min1.name ,min1.reward)
print(selfish.name, selfish.reward)

True
2
25
Fraction 0.08
alpha 0.08536585365853659
m0 0
m1 0
m2 20
m3 20
m4 20
m5 0
m6 10
m7 20
m8 10
m9 30
m10 10
m11 30
m12 30
m13 45
m14 30
selfish 20


In [64]:
show(bc)

## Project 2: Uncles and Uncles Rewards
- Model a Blockchain that uses uncles and uncle rewards
- Model rewarding mechanism to reward uncle block creators
- Model selfish mining (only one attacker) in this blockchain

Try to answer the following questions with your experiment:
1. How do uncles improve the fairness of the blockchain? For this, you should compare the outcome of miners with and without uncles.
2. What is the impact of uncles on selfish mining? Is selfish mining more profitable with uncles?
3. What does it mean in this model for the selfish mining attack to be profitable?

 Since more miners will get rewards for mining, even if they don't finish a block that is part of the longest chain, they will be rewarded for it. So the time and energy used for that block will not be as wasted, and it seems more rewarding to mine. Show net money with and without uncle rewards to display point. The concept of it is for fairness, but may encourage selfish mining. 